In [1]:
import os
import numpy as np
# Train a CNN classifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import losses,activations,optimizers
from matplotlib import pyplot as plt
from PIL import Image, ImageOps
from tensorflow.keras.preprocessing.image import ImageDataGenerator


the train/vaild data must be in the form:
catsdogs ---
    train ---
        cats
        dogs
    valid ---
        cats
        dogs


In [2]:
print('cwd=',os.getcwd())
# Resize image to size proportinally. size is a tuple (width,height)
def resize_image(img, size):
    
    # Convert RGBA images to RGB
    if np.array(img).shape[2] == 4:
        img = img.convert('RGB')
        
    # resize the image
    img.thumbnail(size, Image.ANTIALIAS)
    newimg = Image.new("RGB", size, (255, 255, 255))
    newimg.paste(img, (int((size[0] - img.size[0]) / 2), int((size[1] - img.size[1]) / 2)))
    
    return newimg


cwd= C:\Users\XRUI


In [3]:
train_datagen = ImageDataGenerator(
    rescale=1./255,   # normalization
    # shear_range就是错切变换，让所有点的x坐标(或者y坐标)保持不变，而对应的y坐标(或者x坐标)则按比例发生平移，且平移的大小和该点到x轴(或y轴)的垂直距离成正比。
    shear_range=0.2,
    # 用户指定旋转角度范围，其参数只需指定一个整数即可，但并不是固定以这个角度进行旋转，而是在 [0, 指定角度] 范围内进行随机角度旋转
#    rotation_range=30,
    # width_shift_range & height_shift_range 分别是水平位置平移和上下位置平移，其参数可以是[0, 1]的浮点数，也可以大于1，其最大平移距离为图片长或宽的尺寸乘以参数
#    width_shift_range=0.2,
    zoom_range=0.2,   # 在长宽两个方向进行同等(或不同)程度的放缩操作，参数大于0小于1时，执行的是放大操作
    horizontal_flip=True)  # horizontal flip transform

valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '../../catsdogs/train',
    target_size=(128, 128),
    batch_size=64,
    # the directory must already exists. all generated samples(batch_size*steps_per_epoch*epochs) will be saved in this directory
#    save_to_dir="generated_train_data",
    class_mode='categorical')

validation_generator = valid_datagen.flow_from_directory(
    '../../catsdogs/valid',
    target_size=(128, 128),
    batch_size=64,
    # the directory must already exists
#    save_to_dir="generated_valid_data",
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    '../../catsdogs/test',
    target_size=(128, 128),
    batch_size=1,
    shuffle=False,
    class_mode='categorical')


FileNotFoundError: [WinError 3] 系统找不到指定的路径。: '../../catsdogs/train'

In [ ]:
classnames = np.array(['cat','dog'])
STEP_SIZE_TRAIN = train_generator.n//train_generator.batch_size
STEP_SIZE_VALID = validation_generator.n//validation_generator.batch_size
STEP_SIZE_TEST = test_generator.n//test_generator.batch_size
print('STEP_SIZE_TRAIN =',STEP_SIZE_TRAIN)
print('STEP_SIZE_VALID =',STEP_SIZE_VALID)
print('STEP_SIZE_TEST =',STEP_SIZE_TEST)


In [ ]:
model = Sequential()
# filter size (3,3) step default is (1,1). input shape=(128,128,3) padding default 'valid' (no padding)
# output feature maps: (n+2p-f)/s+1 = (128+0-3)/1+1 = 126. so shape -> (126,126,32)
# parameters number = 3*3*3*32+32 = 896. filter size (3*3*3), filter number = 32
model.add(Conv2D(32, (3, 3), input_shape=(128, 128, 3), activation='relu'))
#model.add(Conv2D(32, (3, 3), strides=(2,2), padding='valid', input_shape=(128,128,3), activation='relu'))
# output feature maps: pool_size=(2,2) so width and height half changed shape -> (63,63,32)
# parameters numbers = 0(no convolution filter)
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
# output feature maps: size=63+0-2+1  shape -> (61,61,32) 
# parameters number = 3*3*32*32+32 = 9248. pay attention the depth of the this layers' filter is the same as the last layer's filter number(32 here)
model.add(Conv2D(32, (3, 3), activation='relu'))
# output feature maps: width and height half changed shape -> (30,30,32)
# parameters numbers = 0(no convolution filter)
model.add(MaxPooling2D(pool_size=(2, 2)))    # strides default to pool_size
# faeture maps no change. parameters = 0
model.add(Dropout(0.5))
# output feature maps: size=30+0-3+1  shape -> (28,28,32) parameter numbers = 3*3*32*32+32 = 9248
model.add(Conv2D(32, (3, 3), activation='relu'))
# output feature maps: width and height half changed. shape -> (14,14,32). no parameters
model.add(MaxPooling2D(pool_size=(2, 2)))
# output feature maps: no change, parameters = 0
model.add(Dropout(0.5))
# output feature maps: shape flattened from (14,14,32) to (14*14*32,) = (6272,)
model.add(Flatten())
# output feature maps: 2, parameters numbers = 2*6272+2 = 12546
model.add(Dense(len(classnames), activation='softmax'))
model.summary()


In [ ]:
optimizer = optimizers.Adam
loss = losses.categorical_crossentropy
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

num_epochs = 100

# steps_per_epoch = total_samples/batch_size
# here the generator generates 32(or 64, ..) samples for a batch, and 50 steps will end far a epoch
history = model.fit(
    train_generator,
    steps_per_epoch=STEP_SIZE_TRAIN,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=STEP_SIZE_VALID)


In [ ]:
# plot training accuracy and loss from history
plt.figure(figsize=(15,10))
plt.subplot(121)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')

plt.subplot(122)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')

plt.show()


In [ ]:
# load predict images
predict_data = []
predict_images = []
size = (128,128)
files = os.listdir('../catsdogs/test')
for f in files:
    if not 'jpg' in f:
        continue
    img = Image.open(os.path.join('../catsdogs/test',f))
    # resize images
    img = resize_image(img,size)
    predict_data.append(np.array(img))



In [ ]:
#img = Image.open('resized_images/hardshell_jackets_test/resized_10269570x1012905_zm.jpeg')
# normalize the image data
predict_data = np.array(predict_data,dtype='float')/255.
# use the model to predict
predicted_labels_encoded = model.predict(predict_data)
print('predicted_labels_encoded =',predicted_labels_encoded)
# predicted_labels_encoded is an array ,so np.argmax must specify the axis, so the result is also an arrar                                 
predicted_labels = np.argmax(predicted_labels_encoded,axis=1) 
print('predicted digits={}'.format(predicted_labels))                                
#predicted_labels = encoder.inverse_transform(predicted_labels)
#print('predicted labels=',predicted_labels)



In [ ]:
# predict using data generator
predicted_labels_encoded2 = model.predict(test_generator,steps=STEP_SIZE_TEST)
print('predicted_labels_encoded2 =',predicted_labels_encoded2)
predicted_labels2 = np.argmax(predicted_labels_encoded2,axis=1) 
print('predicted digits2={}'.format(predicted_labels2)) 



In [ ]:
i = 0
#fig = plt.figure(figsize=(10, 10))
for data in predict_data:
    ax = plt.subplot(1,4,i+1)
    # image data should be converted back to its original value in order to show image
    data = (data * 255.).astype(np.uint8)
    imgplot = plt.imshow(data)
    # classnames is a dictionary [0:'cats',1:'dogs']
    ax.set_title(classnames[predicted_labels[i]])
    i = i+1
plt.show()
